### Importing required libraries.

The data is collected from yahoo finance on a daily basis which can be altered depending on the requirement . 

In [33]:
#Installing Yahoo Finance
#!pip install yfinance
#!pip install pandas_datareader
#!pip install Ta-Lib
#!brew install ta_lib
#!pip install tensortrade[talib]
!pip install scikeras

In [37]:
import pandas as pd
import seaborn as sns
import joblib
import time, datetime
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import yfinance as yf
import pandas_datareader.data as pdr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

### Importing Data from Yahoo Finance.

In [3]:
#Collecting desired ticker from user
ticker = input("Please enter your desired ticker")
start_time = datetime.datetime(1980,1,1)
end_time = datetime.datetime.now().date()

Please enter your desired ticker AAPL


In [4]:
def get_data(ticker):
    df = pdr.get_data_yahoo(ticker, start=start_time, end=end_time)         
    return df

In [5]:
def compute_ema(df):
    df['EMA50'] = df['Close'].ewm(span=50, adjust=False).mean()
    #Setting adjust to False to specify that recursive calculation mode is required.
    df['EMA100'] = df['Close'].ewm(span=100, adjust=False).mean()
    #df['EMA150'] = EMA(df['Close'].values, timeperiod=150)
   # df['EMA200'] = EMA(df['Close'].values, timeperiod=200)
    return df


In [ ]:
#Computing the MACD indicator
#df =computeMACD(df, 12, 26, 9)
#The values can be altered along with the model. 
#The values are normalised between -1 to 1 which is not suitable for the current model
#As we are looking for the intersection of price and MACD.

### Creating the feature variables.

In [6]:
def calculated_features(df):
    df['aboveEMA50'] = np.where(df['Close'] > df['EMA50'], 1, 0)
    df['aboveEMA100'] = np.where(df['Close'] > df['EMA100'], 1, 0)
    return df

In [7]:
# create a new column in the DF based on the conditions
def defined_conditions(df):
    # List of conditions
    condition = [(df['aboveEMA50'] == df['aboveEMA100']) & (df['aboveEMA50'] ==1),
                 (df['aboveEMA50'] == df['aboveEMA100']) & (df['aboveEMA50'] ==0),
                 (df['aboveEMA50'] != df['aboveEMA100'])]
    # List of values to return
    choice  = [1,0,2]

    df['Buy/Sell'] = np.select(condition, choice, "ERROR")
    
    return df
#Considering it as a buy if the price is over 50 ema and 100 ema.
#Considering it as a Sell if the price is under 50 ema and 100 ema. 
#Further analysis is required if the price is over 50 ema and under ema 100 or vice versa.

### Creating DF for multiple stocks and Indices.

In [8]:
# Requesting the stock Data from the user. 
#tickers = ['AAPL','GOOG','AMD','AMZN']

In [21]:


#Calling the get_data function definition. 
df = get_data(ticker)
df = df.reset_index()
df = df.set_index('Date')
#Calculating EMA 50 and 100
df = compute_ema(df)
#Calculating if the signal is to buy or sell.
df = calculated_features(df)
# Rules for placind a trade.
df = defined_conditions(df)


### Seperating Labels and Features from the Data frame and splitting the data

In [22]:
X = df.drop(columns =['Buy/Sell'])
#Reviewing features Data Frame.
X.head(5)

,High,Low,Open,Close,Volume,Adj Close,EMA50,EMA100,aboveEMA50,aboveEMA100
Date,,,,,,,,,,
1980-12-12,0.128906,0.128348,0.128348,0.128348,469033600.0,0.099874,0.128348,0.128348,0,0
1980-12-15,0.122210,0.121652,0.122210,0.121652,175884800.0,0.094663,0.128085,0.128215,0,0
1980-12-16,0.113281,0.112723,0.113281,0.112723,105728000.0,0.087715,0.127483,0.127909,0,0
1980-12-17,0.116071,0.115513,0.115513,0.115513,86441600.0,0.089886,0.127014,0.127663,0,0
1980-12-18,0.119420,0.118862,0.118862,0.118862,73449600.0,0.092492,0.126694,0.127489,0,0


In [23]:
y = df['Buy/Sell']
#Reviewing Label.
y.head(5)

Date
1980-12-12    0
1980-12-15    0
1980-12-16    0
1980-12-17    0
1980-12-18    0
Name: Buy/Sell, dtype: object

In [26]:
# Split the data into training and testing datasets
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state =1)

### Passing the Data to NN to establist a baseline accuracy.

#### Normalising the data using Standard Scaler

In [41]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# encoding class labels as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

#### Adding Layers to Neural Network

In [ ]:
def create_model():
	# create model
	model = Sequential()
	model.add(Dropout(0.2, input_shape=(10,)))
	model.add(Dense(10, activation='relu', kernel_constraint=MaxNorm(3)))
	model.add(Dense(5, activation='tanh', kernel_constraint=MaxNorm(3)))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	sgd = SGD(learning_rate=0.1, momentum=0.9)
	model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
	return model
 
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_model, epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Visible: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))